<h1>Home Assignment injunctions!</h1>
<li>No Loops. Use only elementwise operations</li>
<li>Your code should work for any data in the provided format. Data values (example, customer ids, product ids, number of customers, etc.) should NEVER appear in your code!</li>

<h1>Problem 1:</h1>
Write a function that reads timeseries pricing data from a file into a pandas dataframe and then groups the data as follows:
<li>The arguments to the function are the filename and a threshold number
<li>The function reads the data in the file and creates a new column "pct_change" with the one day percent change
<li>Then groups the data into four categories:
<ul>
<li>"High+" if the percent change is greater than the threshold 
<li>"Low+" if the percent change is zero or positive and less than or equal to the threshold 
<li>"Low-" if the percent change is negative but greater than or equal to -1 * the threshold
<li>"High-" if the percent change is less than -1 * the threshold
</ul>
<li>The function should return a dataframe that contains three columns (count, mean, stdev) and four index values (High+, High-, Low+, Low-)
    <p><b>Note: </b>we have to deal with nan percent changes. Make sure that you don't count a NaN in any of the four categories! (see https://pandas.pydata.org/docs/reference/api/pandas.isna.html)  <p>
For the sample data your function should return a dataframe with the following values for a threshold of 1.0:

<pre>
        count	mean	stdev
High+	63	   2.201357	1.403010
High-	58	  -2.227977	1.133905
Low+	70	   0.466946	0.292076
Low-	60	  -0.506772	0.278789

</pre>

    

<h3>read_csv</h3>
The pandas <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html">read_csv</a> function reads data from a delimited file into a pandas dataframe.



<h2>Sample data</h2>
Use the attached AAPL.csv file

In [1]:
import pandas as pd
import numpy as np
def change_groups(datafile,threshold):
    # read in datafile
    data_df = pd.read_csv(datafile)

    # create pct_change column
    data_df["pct_change"] = data_df["Adj Close"].pct_change(1) * 100

    # define custom grouping function
    pct_groupfunc = lambda row: (
        "High+" if data_df.loc[row, 'pct_change'] > threshold else
        "Low+" if 0 <= data_df.loc[row, 'pct_change'] <= threshold else
        "Low-" if -1 * threshold <= data_df.loc[row, 'pct_change'] < 0 else
        "High-"
    )

    # drop rows where 'pct_change' is NaNs 
    data_df = data_df.dropna(subset=["pct_change"])

    # group by custom function
    pct_groups = data_df.groupby(pct_groupfunc)

    # create summary dataframe
    summary_df = pd.DataFrame({
        "count": pct_groups["pct_change"].count(),
        "mean": pct_groups["pct_change"].mean(),
        "stdev": pct_groups["pct_change"].std(),
    })
    return summary_df

In [2]:
#Test your code
change_groups("Resources/AAPL.csv",1.0)

,count,mean,stdev
High+,63,2.201357,1.403010
High-,58,-2.227977,1.133905
Low+,70,0.466946,0.292076
Low-,60,-0.506772,0.278789


<h1>Problem 2</h1>
A manufacturer has data on orders from customers and product prices in two dataframes (see below). They want to use this data to answer the following questions:
<ol>
    <li>Which customer is responsible for the most revenue</li>
    <li>Which customer is responsible for the highest profit</li>
    <li>Which product is responsible for the highest (dollar) profit</li>
    <li>Which customer and product combination is responsible for the most orders</li>
</ol>
<p>
Obviously, your code should work for any actual data values and pandas dataframes of any length!
<p>For the data below: your answers should be:
    
<pre>
Customer with most profit: 005
Customer with most revenue: 007
Product with most profit: 011
Customer 001 with product 010 with 4 orders is the most ordered customer product pair
</pre>


<h2>Useful functions:</h2>
<li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html">pd.sort_values</a> </li>
<li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html">pandas dataframe join</a> the last example on the linked page is probably what you need here!</li>
<li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html">pd.groupby</a></li>
<li><a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html">pandas join</a></li>

In [3]:
import pandas as pd
import numpy as np
order_data = pd.DataFrame([["001","010",24],
                           ["007","012",35],
                           ["001","011",15],
                           ["005","010",30],
                           ["007","011",17],
                           ["005","011",81],
                           ["001","010",32],
                           ["007","012",89],
                           ["001","010",16],
                           ["001","010",33]],columns=["customer","product","amount"])
products = pd.DataFrame([['010',22.3,17.2],
                        ['011',11.7,5.5],
                        ['012',62.5,61.4]],columns=['product','price','cost'])                     

In [4]:
order_product = order_data.merge(products,how="left",on="product")
order_product["revenue"] = order_product["amount"] * order_product["price"]
order_product["profit"] = order_product["revenue"] - order_product["amount"] * order_product["cost"]

In [5]:
print(f"Customer with most profit: {order_product.groupby('customer').sum()['profit'].idxmax()}")
print(f"Customer with most revenue: {order_product.groupby('customer').sum()['revenue'].idxmax()}")
print(f"Product with most profit: {order_product.groupby('product').sum()['profit'].idxmax()}")
print(f"Customer {order_product.groupby(['customer','product']).count()['amount'].idxmax()[0]} with product {order_product.groupby(['customer','product']).count()['amount'].idxmax()[1]} with {order_product.groupby(['customer','product']).count()['amount'].max()} orders is the most ordered customer product pair")

Customer with most profit: 005
Customer with most revenue: 007
Product with most profit: 011
Customer 001 with product 010 with 4 orders is the most ordered customer product pair


<h1>Problem 3</h1>
In this problem you'll get some practice getting and combining data from the St. Louis Federal Reserve (FRED). Get the following data from FRED (01/01/2010 to 12/31/2022):

<pre>
"TB3MS" #3 month t-bill market yield 
"DGS10" #10 year constant maturity government bond market yield
"NB000334Q" #Real GDP index quarterly (index = 100 at 2012)
"CPIAUCSL" #Consumer price index for all urban consumers seasonally adjusted
</pre>

You will need to register with the federal reserve and get an API key. 

1. Create a new account at https://fredaccount.stlouisfed.org/login/secure/
2. Go to My Account (top right corner) once you're logged in and select the Api-key option. Copy the API key and save it in a variable in your notebook

Since these data items have different frequencies (some are daily, some monthly, some quarterly), make separate fred api calls for each. 

Then, as a proxy for the stock market, get data for the ticker SPY,the S&P ETF, from yfinance. 

Align all the data to the end of the business quarter (i.e., use the value on the last day of each quarter).

For the ETF, calculate one day percent changes and shift the data back by one quarter (we're interested in the correlation between macroeconomic data in one quarter and the performance of the S&P in the next quarter). For example, if the quarterly percentage change on 3/31 is 5% and on 6/30 is 2.5% (i.e., the change in value between 3/31 and 6/30), we want to align the percent change on 6/30 with the the macroeconomic data as of 3/31. So we need to replace the data on 3/31 by 2.5%

Using the pandas join function, join all the data into one dataframe with the quarter end date as the index

Generate the correlation matrix. This is what you should get:

<pre>
            TB3MS	     DGS10	    NB000334Q	 CPIAUCSL	   SPY
TB3MS   	1.000000	0.409817	0.063098	0.049515	-0.079082
DGS10	    0.409817	1.000000	0.056068	0.060827	-0.329453
NB000334Q	0.063098	0.056068	1.000000	0.162265	-0.009601
CPIAUCSL	0.049515	0.060827	0.162265	1.000000	-0.398615
SPY	        -0.079082	-0.329453	-0.009601	-0.398615	1.000000

</pre>

<h3>Notes:</h3>

1. positive numbers will shift forward while negative numbers will shift backward

    
2. To rename a column, use df.rename(columns={"old_name":"new_name"})

3. To join two dataframes use:
    new_df = old_df1.join(old_df2)

<h3>Install fredapi</h3>

In [6]:
# !pip install fredapi

<h3>Using the api</h3>
<li>Make sure you have the api key</li>
<li>Then use the example below to get data</li>

In [7]:

with open("credentials/fred.txt") as f:
    API_KEY = f.read().strip()

In [8]:
#Import fredapi and datetime
import fredapi as fa
import datetime as datetime

#Copy your key into 
#Create a fred object that knows the api key
#Each request will then automatically contain that key
fred = fa.Fred(api_key=API_KEY)

#Get a data series
#For example, to get the TB3MS series use
start = datetime.datetime(month=1,day=1,year=2010)
end=datetime.datetime(month=12,day=31,year=2022)

fred.get_series("TB3MS",observation_start=start,observation_end=end)

2010-01-01    0.06
2010-02-01    0.11
2010-03-01    0.15
2010-04-01    0.16
2010-05-01    0.16
              ... 
2022-08-01    2.63
2022-09-01    3.13
2022-10-01    3.72
2022-11-01    4.15
2022-12-01    4.25
Length: 156, dtype: float64

<h3>Problem setup</h3>

In [9]:
import fredapi as fa
fred = fa.Fred(api_key=API_KEY)

#Date range
import datetime
import numpy as np
start = datetime.datetime(month=1,day=1,year=2010)
end = datetime.datetime(month=12,day=31,year=2022)

#DATA IDENTIFIERS
tbill3 = "TB3MS" #3 month t-bill market yield 
tnote10 = "DGS10" #10 year constant maturity market yield
gdp_index = "NB000334Q" #Real GDP index quarterly (index = 100 at 2012)
cpi = "CPIAUCSL" #Consumer price for all urban consumers seasonally adjusted

import yfinance as yf
#Setting up SPY dataframe
spy_df = yf.download("SPY", start=start, end=end)["Adj Close"].pct_change().resample('Q').last().shift(-1).to_frame()
spy_df.rename(columns={"Adj Close":"SPY"}, inplace=True)

#Setting up tbill3 dataframe
tbill3_df = fred.get_series(tbill3, observation_start=start, observation_end=end).resample('Q').last().to_frame(tbill3)

#Setting up tnote10 dataframe
tnote10_df = fred.get_series(tnote10, observation_start=start, observation_end=end).resample('Q').last().to_frame(tnote10)

#Setting up gdp_index dataframe
gdp_index_df = fred.get_series(gdp_index, observation_start=start, observation_end=end).resample('Q').last().to_frame(gdp_index)

#Setting up cpi dataframe
cpi_df = fred.get_series(cpi, observation_start=start, observation_end=end).resample('Q').last().to_frame(cpi)

[*********************100%%**********************]  1 of 1 completed


In [10]:
combined_df = tbill3_df.join(tnote10_df).join(gdp_index_df).join(cpi_df).join(spy_df)

combined_df.corr()

,TB3MS,DGS10,NB000334Q,CPIAUCSL,SPY
TB3MS,1.000000,0.409817,0.594268,0.568997,0.056633
DGS10,0.409817,1.000000,-0.166219,-0.081883,-0.186375
NB000334Q,0.594268,-0.166219,1.000000,0.924669,-0.143251
CPIAUCSL,0.568997,-0.081883,0.924669,1.000000,-0.184764
SPY,0.056633,-0.186375,-0.143251,-0.184764,1.000000
